In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical

In [ ]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


291/291 - 50s - loss: 0.8292 - accuracy: 0.6393 - 50s/epoch - 171ms/step
144/144 - 5s - loss: 0.7593 - accuracy: 0.6662 - 5s/epoch - 34ms/step
0.7592564821243286
0.666229784488678
['loss', 'accuracy']


In [ ]:
model.save('sentimentAnalysis.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
model= load_model('sentimentAnalysis.h5')

In [ ]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [ ]:
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0]
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 314ms/epoch - 314ms/step
[0.46129748 0.12149629 0.41720623]
Neutral


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20, 40]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

<ipython-input-7-5b4d4aa083bb>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2)


744/744 - 99s - loss: 0.8293 - accuracy: 0.6480 - 99s/epoch - 133ms/step
186/186 - 3s - loss: 0.7483 - accuracy: 0.6697 - 3s/epoch - 17ms/step
744/744 - 100s - loss: 0.8195 - accuracy: 0.6492 - 100s/epoch - 135ms/step
186/186 - 4s - loss: 0.7616 - accuracy: 0.6708 - 4s/epoch - 20ms/step
744/744 - 100s - loss: 0.8219 - accuracy: 0.6464 - 100s/epoch - 134ms/step
186/186 - 3s - loss: 0.7582 - accuracy: 0.6740 - 3s/epoch - 14ms/step
744/744 - 100s - loss: 0.8261 - accuracy: 0.6465 - 100s/epoch - 134ms/step
186/186 - 3s - loss: 0.7562 - accuracy: 0.6690 - 3s/epoch - 14ms/step
744/744 - 99s - loss: 0.8213 - accuracy: 0.6437 - 99s/epoch - 133ms/step
186/186 - 3s - loss: 0.7806 - accuracy: 0.6706 - 3s/epoch - 14ms/step
Epoch 1/2
744/744 - 100s - loss: 0.8243 - accuracy: 0.6470 - 100s/epoch - 134ms/step
Epoch 2/2
744/744 - 96s - loss: 0.6831 - accuracy: 0.7092 - 96s/epoch - 129ms/step
186/186 - 3s - loss: 0.7271 - accuracy: 0.6934 - 3s/epoch - 14ms/step
Epoch 1/2
744/744 - 99s - loss: 0.8190 - 